In [5]:
import warnings
warnings.filterwarnings('ignore')

from model import myVGG19
import gradio as gr
import torch
from torchvision import transforms

import sys
import torchvision
print("python version :", sys.version)
print("gradio version :", gr.__version__)
print("torch version :", torch.__version__)
print("torchvision version :", torchvision.__version__)

python version : 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
gradio version : 4.25.0
torch version : 2.2.1+cu121
torchvision version : 0.17.1+cu121


In [6]:
def get_test_image(img):
    # 사진 전처리를 위한 변환 정의
    transform = transforms.Compose([
        transforms.Resize((48, 48)),  # 예제에서 사용하는 모델 입력 크기에 맞춰 조정
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5076], std=[0.2120]),
    ])

    # 사진 불러오기 및 전처리
    img_transformed = transform(img)
    img_transformed = img_transformed.unsqueeze(0)  # 모델에 입력하기 위해 배치 차원 추가

    return img_transformed


# Gradio 인터페이스에서 사용할 함수 정의
def predict_emotion(img):
    # test image 처리
    img_transformed = get_test_image(img)
    model = torch.load('./vgg19_model.pt')  # 모델 변경 시 수정!!!!!! 해당 모델 class import도 새로 해야 됨

    # 모델을 평가 모드로 설정하고 사진 분류
    model.eval()
    with torch.no_grad():
        output = model(img_transformed.to('cuda' if torch.cuda.is_available() else 'cpu'))
        _, predicted = torch.max(output, 1)

    # 감정 레이블 정의 (컬럼 순서에 따라)
    emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt', 'unknown', 'NF']

    # 예측된 클래스 인덱스
    predicted_class_index = predicted.item()

    # 예측된 감정 레이블
    predicted_label = emotion_labels[predicted_class_index]

    return predicted_label

In [7]:
gr.Interface(fn=predict_emotion,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=10),
             theme="HaleyCH/HaleyCH_Theme").launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
